### Day 20: Race Condition

https://adventofcode.com/2024/day/20

In [1]:
from pathlib import Path
import sys

sys.path.append("..")

from tools import get_input, profile

tst = [[c for c in line] for line in (Path().parent / "test.txt").read_text(encoding="utf-8").splitlines()]
inp = [[c for c in line] for line in get_input(20).splitlines()]

In [2]:
from collections import defaultdict


def find_start(data: list[list[str]]) -> tuple[int, int]:
    for x in range(len(data)):
        for y in range(len(data[0])):
            if data[x][y] == "S":
                return x, y
        
def mark_track(data: list[list[str]], s: tuple[int, int]) -> list[tuple[int, int]]:
    path = [s]
    data[s[0]][s[1]] = 0
    while True:
        x, y = path[-1]
        for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
            nx = x + dx
            ny = y + dy
            if 0 <= nx < len(data) and 0 <= ny < len(data[0]):
                if data[nx][ny] == "E":
                    data[nx][ny] = len(path)
                    path.append((nx, ny))
                    return path
                if data[nx][ny] == ".":
                    data[nx][ny] = len(path)
                    path.append((nx, ny))
                    break
        
def solution_1(data: list[list[str]]) -> dict[int, int]:
    track = [line.copy() for line in data]
    s = find_start(data)
    path = mark_track(track, s)
    shortcuts = defaultdict(int)
    for i, (x, y) in enumerate(path):
        for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
            nx = x + dx
            ny = y + dy
            if 0 <= nx < len(track) and 0 <= ny < len(track[0]) and track[nx][ny] == "#":
                nnx = nx + dx
                nny = ny + dy
                if 0 <= nnx < len(track) and 0 <= nny < len(track[0]) and track[nnx][nny] != "#":
                    d = track[nnx][nny] - i - 2
                    if d > 0:
                        shortcuts[d] += 1
    return shortcuts

assert sum(v for k, v in solution_1(tst).items() if k >= 8) == 14
sum(v for k, v in solution_1(inp).items() if k >= 100)  # 1417


1417

In [3]:
def solution_2(data: list[list[str]], limit: int, cheat: int) -> int:
    track = [line.copy() for line in data]
    s = find_start(data)
    path = mark_track(track, s)[::-1]
    res = 0
    while path:
        x, y = path.pop()
        for xx, yy in path:
            d = abs(x - xx) + abs(y - yy)
            if d <= cheat and track[xx][yy] - track[x][y] - d >= limit:
                res += 1
    return res

assert solution_2(tst, 8, 2) == 14
assert solution_2(tst, 76, 20) == 3
assert solution_2(tst, 70, 20) == 41
solution_2(inp, 100, 20)  # 1014683

1014683

In [ ]:
def solution_2_loop(data: list[list[str]], limit: int, cheat: int) -> int:
    track = [line.copy() for line in data]
    s = find_start(data)
    path = mark_track(track, s)[::-1]
    res = 0
    while path:
        x, y = path.pop()
        for i in range(-cheat, cheat + 1):
            for j in range(-cheat + abs(i), cheat + 1 - abs(i)):
                if (
                    0 <= x + i < len(track)
                    and 0 <= y + j < len(track[0])
                    and track[x + i][y + j] != "#"
                    and track[x + i][y + j] - track[x][y] - abs(i) - abs(j) >= limit
                ):
                    res += 1
    return res

assert solution_2_loop(tst, 70, 20) == 41
solution_2_loop(inp, 100, 20)  # 1014683